In [90]:
import json
import pandas as pd
import re
import random
import dice
import jsonpickle

import jsonpickle.ext.pandas as jsonpickle_pandas
jsonpickle_pandas.register_handlers()

In [116]:
class RangeDict(dict):
    def __getitem__(self, item):
        if not isinstance(item, range):
            for key in self:
                if item in key:
                    return self[key]
            raise KeyError(item)
        else:
            return super().__getitem__(item)

        
class Plunder_Table:
    # A collection of multiple entries via dict keys
    def __init__(self, entries=None, entry_keys=None, examples=None):
        self.entries = {key: entry for key,entry in zip(entry_keys, entries)}
        self.examples = {key: example for key,example in zip(entry_keys, examples)}
        
    def __str__(self):
        return "\n".join([f"{key} {entry}" for key,entry in self.entries.items()])
    
    def roll(self, roll):
        return self.entries[roll]
        
        
class Table:
    # A collection of multiple entries via range dicts
    def __init__(self, entries=None, entry_ranges=None):
        self.entries = RangeDict({range(key[0], key[1]): entry for key,entry in zip(entry_ranges, entries)})
        
    def __str__(self):
        return "\n".join([f"{key} {entry}" for key,entry in self.entries.items()])
    
    def roll(self, roll):
        return self.entries[roll]

    
class Entry:
    # A collection of multiple items
    def __init__(self, items):
        self.items = items.copy()
        
    def __str__(self):
        return ", ".join([str(item) for item in self.items])

    
class Item:
    def __init__(self, name, chance=None, num_items=1, table_roll=None, condition=None):
        # name can be item name or table reference
        self.name = name
        self.chance = chance
        self.num_items = num_items
        self.table_roll = table_roll
        self.condition = condition
        
    def __str__(self):
        return (f'{self.num_items}{" "+self.condition if self.condition is not None else ""} '
                f'{self.name}{" "+self.table_roll if self.table_roll is not None else ""}')
        
    def collapse_rolls():
        # return new item in collapsed form
        # roll num items if not int
        # roll on tables for each item
        pass

In [117]:
items = [Item(k) for k in ["gun","sword","knife"]]
entry1 = Entry(items[:1])
entry2 = Entry(items[1:])
entry3 = Entry(items)

entries = [entry1, entry2, entry3]
entry_ranges = [(1,5),(6,7),(8,8)]

tab = Table(entries=entries, entry_ranges=entry_ranges)
print(tab)

range(1, 5) 1 gun
range(6, 7) 1 sword, 1 knife
range(8, 8) 1 gun, 1 sword, 1 knife


In [118]:
# Roll decoded string from plunder table
word_numbers = {
    "one":1,
    "two":2,
}

def decode(plunder, top=False):
    items = []
    #GROUP_REGEX = re.compile(r"^(?:(\d+)\% chance of )?(\d(?:d\d+)?(?:[\+\-]\d+)? )?([^\(]+)(?:\((.+)\))?",re.I)
    #GROUP_REGEX = re.compile(r"^(?:(\d+)\% chance of )?(\d(?:d\d+)?(?:[\+\-]\d+)? )?([^\(\+]+)(?:[\(\+]([^\)]+))?",re.I)
    GROUP_REGEX = re.compile(r"^(?:(\d+)\% chance of )?(\d(?:d\d+)?(?:[\+\-]\d+)? )?(.{2}[^\(\+]+)(?:[\(\+]([^\)]+))?",re.I)
    # 1: percent chance of getting, 2: xdx+x of , 3: item on table with roll, 4: (xdx+x) 
    
    
    if top:
        looper = plunder.split(",")
    else:
        looper = [plunder]

    for item in looper:
        item = item.strip()
        #try:
        match = GROUP_REGEX.search(item)
        if match is not None:
            #drops.append(f"{match.group(1)}, {match.group(2)}, {match.group(3)}, {match.group(4)}")
            num_items = match.group(2).strip() if match.group(2) is not None else 1
            
            # If name starts with one or two etc...
            name = match.group(3).strip()
            if match.group(3).split()[0].strip().lower() in word_numbers:
                split = match.group(3).split(" ", 1)
                name = split[1].strip()
                num_itmes = num_items * word_numbers[match.group(3).split()[0].strip().lower()]
            name = name.lower().rstrip("s")
            
            table_roll = match.group(4)
            if name == "random loot":
                table_roll = "1d20"+("+"+str(table_roll) if table_roll is not None else "")
            table_roll = str(table_roll) if table_roll is not None else table_roll
            chance = match.group(1)
            chance = str(chance) if chance is not None else chance
            
            items.append(Item(str(name), chance=chance, num_items=str(num_items), table_roll=table_roll, condition=None))
    
    entry = Entry(items)        
            
    return entry

In [119]:
df_plunder = pd.read_csv("tables/plunder.csv").iloc[:, 1:]
print(df_plunder.columns)

entries = []
for line in df_plunder[" Plunder"]:
    entries.append(decode(line, top=True))
    
p_table = Plunder_Table(entries=entries, entry_keys=df_plunder["Type"], examples=df_plunder["Example"])
print(p_table)

Index(['Type', 'Example', ' Plunder'], dtype='object')
P1 1 weapon 1d6, 1 armor 1d6, 1 ration, 1 random loot 1d20
P2 1 weapon 1d6+4, 1 armor 1d6+4, 1 ration, 1 random loot 1d20+3
P3 1 weapon 1d6+6, 1 armor 1d6+6, 1d4 ration, 1 random loot 1d20+5
P4 1 weapon 1d6+10, 1 armor 1d8+8, 1 random loot 1d20+10
P5 1 weapon 1d6+12, 1 armor 1d6+1, 1 random loot 1d20+5
P6 1 laser pistol, 1 old terran clothing, 1 random loot 1d20+5
P7 1 weapon 1d6+20, 1 armor 1d4+18, 1 random loot 1d20+10
P8 1 energy weapon 1d4+2, 1 armor 1d4+18, 1 random loot 1d20+12
P9 1 energy weapon 1d6+6, 1 armor 1d6+18, 1 random loot 1d20+12
P10 1 energy weapon 1d4+10, 1 armor 1d6+20, 1 random loot 1d20+20
G1 1 weapon 1d10, 1 armor 1d10, 1 random loot 1d20
G2 1d6 energy weapon 1d10+4, 1d6 suits of harmony armor, 1 armor 1d8+18, 1d6 tl4 spare part, 1d6 tl5 spare part, 2d6 type a power cell
G3 1d6 weapon 1d10+2, 1 weapon 1d6+9, 1d6 armor 1d10+3, 1d6 tl2 spare part, 1d4 x 20 rounds of ammo
G4 1d6 weapon 1d10+10, 1d4 energy weapon

In [120]:
frozen = jsonpickle.encode(p_table)
f = open("json/plunder.json", "w")
f.write(frozen)

20594

In [121]:
print(p_table)

P1 1 weapon 1d6, 1 armor 1d6, 1 ration, 1 random loot 1d20
P2 1 weapon 1d6+4, 1 armor 1d6+4, 1 ration, 1 random loot 1d20+3
P3 1 weapon 1d6+6, 1 armor 1d6+6, 1d4 ration, 1 random loot 1d20+5
P4 1 weapon 1d6+10, 1 armor 1d8+8, 1 random loot 1d20+10
P5 1 weapon 1d6+12, 1 armor 1d6+1, 1 random loot 1d20+5
P6 1 laser pistol, 1 old terran clothing, 1 random loot 1d20+5
P7 1 weapon 1d6+20, 1 armor 1d4+18, 1 random loot 1d20+10
P8 1 energy weapon 1d4+2, 1 armor 1d4+18, 1 random loot 1d20+12
P9 1 energy weapon 1d6+6, 1 armor 1d6+18, 1 random loot 1d20+12
P10 1 energy weapon 1d4+10, 1 armor 1d6+20, 1 random loot 1d20+20
G1 1 weapon 1d10, 1 armor 1d10, 1 random loot 1d20
G2 1d6 energy weapon 1d10+4, 1d6 suits of harmony armor, 1 armor 1d8+18, 1d6 tl4 spare part, 1d6 tl5 spare part, 2d6 type a power cell
G3 1d6 weapon 1d10+2, 1 weapon 1d6+9, 1d6 armor 1d10+3, 1d6 tl2 spare part, 1d4 x 20 rounds of ammo
G4 1d6 weapon 1d10+10, 1d4 energy weapon 1d8, 1d6 suits of insurgent combat shell, 1d6 tl3 spar

In [131]:
def get_entry_range(string):
    string = string.strip()
    string = string.strip("+")
    string = string.split("-")
    string[-1] = 100 if string[-1] == "00" else string[-1]
    return (int(string[0]),int(string[-1])+1)

In [138]:
# import csv's
df_armor = pd.read_csv("tables/armor.csv").iloc[:, 1:]
df_common_items = pd.read_csv("tables/common_items.csv").iloc[:, 1:]
df_energy_weapon = pd.read_csv("tables/energy_weapon.csv").iloc[:, 1:]
df_equipment_wear = pd.read_csv("tables/equipment_wear.csv").iloc[:, 1:]
df_item_quirks = pd.read_csv("tables/item_quirks.csv").iloc[:, 1:]
df_plunder = pd.read_csv("tables/plunder.csv").iloc[:, 1:]
df_random_loot = pd.read_csv("tables/random_loot.csv").iloc[:, 1:]
df_rare_items = pd.read_csv("tables/rare_items.csv").iloc[:, 1:]
df_uncommon_items = pd.read_csv("tables/uncommon_items.csv").iloc[:, 1:]
df_weapon = pd.read_csv("tables/weapon.csv").iloc[:, 1:]

tables = {
    "armor":df_armor,
    "common item":df_common_items,
    "energy weapon":df_energy_weapon,
    "equipment wear":df_equipment_wear,
    "item quirk":df_item_quirks,
    #"plunder":df_plunder,
    "random loot":df_random_loot,
    "rare item":df_rare_items,
    "uncommon item":df_uncommon_items,
    "weapon":df_weapon
}

#for key,table in tables.items():
if True:
    key = "weapon"
    table = df_weapon
    
    print(key.upper())
    entries = []
    entry_ranges = []
    for index, row in table.iterrows():
        entries.append(decode(row[1], top=False))
        entry_ranges.append(get_entry_range(str(row[0])))
    t_table = Table(entries=entries, entry_ranges=entry_ranges)
    print(t_table)
    #print(t_table)
    #p_table = Plunder_Table(entries=entries, entry_keys=df_plunder["Type"], examples=df_plunder["Example"])
    
    frozen = jsonpickle.encode(t_table)
    f = open(f"json/{'_'.join(key.split())}.json", "w")
    f.write(frozen)

WEAPON
range(1, 2) 1 knife
range(2, 3) 1 club
range(3, 4) 1 spear
range(4, 5) 1 spear
range(5, 6) 1 sword
range(6, 7) 1 axe
range(7, 8) 1 breechloading rifle
range(8, 9) 1 spear
range(9, 10) 1 sword
range(10, 11) 1 revolver
range(11, 12) 1 revolver
range(12, 13) 1 semi-auto pistol
range(13, 14) 1 semi-auto rifle
range(14, 15) 1 shotgun
range(15, 16) 1 monoblade
range(16, 17) 1 submachine gun
range(17, 18) 1 combat rifle
range(18, 19) 1 monoblade
range(19, 20) 1 combat rifle
range(20, 21) 1 combat shotgun
range(21, 22) 1 huge monoblade
range(22, 23) 1 combat rifle
range(23, 24) 1 mag pistol
range(24, 25) 1 mag pistol
range(25, 26) 1 mag rifle
range(26, 27) 1 mag rifle
